In [2]:
import json
from pathlib import Path
import cv2
import numpy as np
from preprocess.utils import mean_preprocess
from config import tiny_image_net_config as config
import tensorflow as tf
import os


model = tf.keras.models.load_model(config.SAVED_MODEL_PATH)

with open(config.DATASET_MEAN) as json_file:
    means = json.load(json_file)

testPaths = []
for i in range(len(os.listdir(config.TEST_PATH))):
    path = config.TEST_PATH/f"test_{i}.JPEG"
    
    testPaths.append(path)
X_test = []

for image in testPaths:
    img = cv2.imread(str(image))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    X_test.append(img)

X_test = np.array(X_test).astype("float64")
X_test[:, :, :, 0] -= means["R"]
X_test[:, :, :, 1] -= means["G"]
X_test[:, :, :, 2] -= means["B"]
json_file.close()
X_test.shape

(10000, 64, 64, 3)

In [3]:
preds = model.predict(X_test, batch_size=64, verbose=1, max_queue_size=64*2)

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [4]:
def one_hot_decode(encoded_seq):
    return [np.argmax(vector) for vector in encoded_seq]

preds = one_hot_decode(preds)
preds[:10]

[172, 150, 44, 152, 22, 148, 149, 105, 54, 150]

In [5]:
testPaths[:10]

[WindowsPath('D:/Projects/ML_projects/tiny_imagenet/tiny-imagenet-200/test/images/test_0.JPEG'),
 WindowsPath('D:/Projects/ML_projects/tiny_imagenet/tiny-imagenet-200/test/images/test_1.JPEG'),
 WindowsPath('D:/Projects/ML_projects/tiny_imagenet/tiny-imagenet-200/test/images/test_2.JPEG'),
 WindowsPath('D:/Projects/ML_projects/tiny_imagenet/tiny-imagenet-200/test/images/test_3.JPEG'),
 WindowsPath('D:/Projects/ML_projects/tiny_imagenet/tiny-imagenet-200/test/images/test_4.JPEG'),
 WindowsPath('D:/Projects/ML_projects/tiny_imagenet/tiny-imagenet-200/test/images/test_5.JPEG'),
 WindowsPath('D:/Projects/ML_projects/tiny_imagenet/tiny-imagenet-200/test/images/test_6.JPEG'),
 WindowsPath('D:/Projects/ML_projects/tiny_imagenet/tiny-imagenet-200/test/images/test_7.JPEG'),
 WindowsPath('D:/Projects/ML_projects/tiny_imagenet/tiny-imagenet-200/test/images/test_8.JPEG'),
 WindowsPath('D:/Projects/ML_projects/tiny_imagenet/tiny-imagenet-200/test/images/test_9.JPEG')]

In [16]:
preds_words = []
with open(config.JSON_CLASS_INDICES) as json_file:
    class_indices = json.load(json_file)
    
for pred in preds:
    for k, v in class_indices.items():
        if pred == int(k):
            preds_words.append(v[0])
             
json_file.close()
        
preds_words[:10]

['n04560804',
 'n04259630',
 'n02279972',
 'n04275548',
 'n02058221',
 'n04251144',
 'n04254777',
 'n03404251',
 'n02480495',
 'n04259630']

In [23]:
testImage = [str(p).split(os.path.sep)[-1] for p in testPaths]
testImage[:10]

['test_0.JPEG',
 'test_1.JPEG',
 'test_2.JPEG',
 'test_3.JPEG',
 'test_4.JPEG',
 'test_5.JPEG',
 'test_6.JPEG',
 'test_7.JPEG',
 'test_8.JPEG',
 'test_9.JPEG']

In [27]:
import pandas as pd

df = pd.DataFrame(data=np.array([testImage, preds_words]).T)
df

,0,1
0,test_0.JPEG,n04560804
1,test_1.JPEG,n04259630
2,test_2.JPEG,n02279972
3,test_3.JPEG,n04275548
4,test_4.JPEG,n02058221
...,...,...
9995,test_9995.JPEG,n01882714
9996,test_9996.JPEG,n02279972
9997,test_9997.JPEG,n04265275
9998,test_9998.JPEG,n03584254


In [29]:
df.to_csv("SUNetID.txt", index=False, header=False)